### Coreference Reformat

**** This notebook has a bug (likely in extract_chains) that causes errors, signified by negative numbers, in the output data. However, none of those errors occur in the data sampled for evaluation, so it is used for now*****

In this notebook, we get the results outputted by coref_faa.py in the data/results folder, and add a column called corefs. This column follows the format expected by the cr_eval.py script in evaluations/quantitative, and so makes evaluation much simpler.

The desired format is like so:\
\[ coreference_chain, coreference_chain, ... ] where coreference_chain = \[mention_span, mention_span, ...] and mention_span = \[start_word_index, end_word_index]\
Such that the coreference chain for the sentence "PILOT LANDED ON WHAT HE THOUGHT TO BE ONE FOOT HIGH GRASS. IT TURNED OUT TO BE THREE FEET HIGH. ACFT NOSED OVER.":\
Which is: \[\["PILOT", "HE"], \["ONE FOOT HIGH GRASS", "IT"]]\
Appears as: [[0,0],[4,4],[[8,11],[13,13]]]\
The word indices are based on the word tokenization used in data/FAA_data/faa.conll, which is the input data for ASP and s2e-coref. The word indices continue to increase throughout the whole doc/entry and do not reset at sentence starts

In [117]:
import pandas as pd
import re

In [118]:
result_df = pd.read_csv('../../tool_results/neuralcoref/FAA_DataModel_small_20240805145715.csv')
result_df.head()

,c5,c119,c119_coref,c119_ner,c119_pos,c119_lemmatized,c119_dependency,c119_sentiment
0,19750315005389A,TAILWHEEL COCKED RIGHT PRIOR TO TKOF. ...,[],[],"[('TAILWHEEL', 'PROPN'), ('COCKED', 'PROPN'), ...","['TAILWHEEL', 'COCKED', 'right', 'PRIOR', 'to'...","[('TAILWHEEL', 'compound', 'COCKED'), ('COCKED...",0.142857
1,19750419011349A,TOW PLANE BECAME AIRBORNE THEN SETTLED.STUDENT...,[],"[('TOW', 'ORG'), ('TOW', 'ORG'), ('TROUBLE & R...","[('TOW', 'PROPN'), ('PLANE', 'NOUN'), ('BECAME...","['TOW', 'plane', 'become', 'AIRBORNE', 'then',...","[('TOW', 'compound', 'PLANE'), ('PLANE', 'ROOT...",-0.200000
2,19751029037799A,"2ND ILS APCH,ACFT'S G/S INOP.LOM TUNED TO WRON...",[],"[('ILS APCH', 'ORG'), ('ACFT', 'ORG'), ('FREQ'...","[('2ND', 'NUM'), ('ILS', 'PROPN'), ('APCH', 'P...","['2nd', 'ILS', 'APCH', ',', 'ACFT', ""'s"", 'G',...","[('2ND', 'nummod', 'APCH'), ('ILS', 'compound'...",-0.250000
3,19751209037899A,PLT NOTED SOFT R BRAKE PEDAL DRG TAXI TO TKOF....,[],[],"[('PLT', 'PROPN'), ('NOTED', 'PROPN'), ('SOFT'...","['PLT', 'NOTED', 'SOFT', 'r', 'BRAKE', 'PEDAL'...","[('PLT', 'nmod', 'R'), ('NOTED', 'advmod', 'R'...",0.127083
4,19750818025579A,TAXI OFF HARD SFC DUE TFC R MAIN GR BROKE THRO...,[],"[('MAIN', 'ORG'), ('WASHED', 'ORG')]","[('TAXI', 'VERB'), ('OFF', 'PART'), ('HARD', '...","['taxi', 'off', 'HARD', 'SFC', 'DUE', 'TFC', '...","[('TAXI', 'ROOT', 'TAXI'), ('OFF', 'prep', 'TA...",-0.083333


**Extract lists of coreferences from the string representation in column c119_coref**

In [105]:
chain_p = re.compile(r'\[([^\[\]]+: )?(\[[^\[\]]+\])?(, [^\[\]]+: )?(\[[^\[\]]+\])?(, [^\[\]]+: )?(\[[^\[\]]+\])?(, [^\[\]]+: )?(\[[^\[\]]+\])?\]') # can capture up to 4 coreference chains
coref_p = re.compile(r'\[(.+)(, )(.+)(, )?(.+)?(, )?(.+)?(, )?(.+)?(, )?(.+)?\]')

def extract_chains(data_in):
        
    # Match coreference chains in c119_coref output
    coref_chains = []
    for match_group in re.match(chain_p, data_in).groups():
        if match_group is not None and '[' in match_group:

            # Match coreference mentions in coreference chain and store as list
            coref_chain = [group for group in re.match(coref_p, match_group).groups() if group is not None and group != ', ']
            coref_chains.append(coref_chain)

    return coref_chains

In [106]:
c119_coref_chains = []
for i in range(len(result_df)):
    coref_chains = extract_chains(result_df['c119_coref'].iat[i])
    c119_coref_chains.append(coref_chains)
c119_coref_chains[2320:2330] # sample output

[[['THE ACFT ENGINE', 'THE ACFT']], [], [], [], [], [], [], [], [], []]

**Get FAA data in format {c5_id:{0: word0, 1: word1, ..., n: wordn}} using word tokenization from faa.conll**

In [107]:
faa = {}

with open('../../OMIn_dataset/data/FAA_data/faa.conll') as f:
    text = f.read()

docs = text.split('#begin document ')

for doc in docs:
    if doc[:5] == '(faa/':
        word_count = 0
        c5_id = doc.split('_')[1][:15]
        faa[c5_id] = {}
        lines = doc.split('\n')
        for line in lines[1:]:
            if 'faa' in line:
                faa[c5_id][word_count] = line.split()[3].upper()
                word_count = word_count + 1

**Get word indices for c119_coref col in result_df**

In [108]:
def get_spans(mentions, words):
    ''' Input:
    - mentions:['MENTION1','MENTION2',...]
    - words: ['This','is','a','sentence','.','This','is','another','sentence','.'] (dict values)
        Output: [[startidx_mention1, end_idxmention1], [startidx_mention2, end_idxmention2], ...]
    '''

    mention_spans = []

    if "'S" in words.values():
        idx = list(words.values()).index("'S")
        words[idx-1] = words[idx-1] + "'S"
        del words[idx]

    bookmark = -1
    
    for imention, mention in enumerate(mentions):

        mention = mention.replace('(', ' ( ').replace(')',' ) ').replace('  ',' ')
        mention = mention.replace(',',' , ').replace('  ',' ')

        mention_span = [-1, -1] # if conditions below aren't met, [-1,-1 is returned]

        tokens = list(words.values())
        idxs = list(words.keys())
        
        start_indices = [i for i in range(len(tokens)) if tokens[i:i+len(mention.split())] == mention.split() and i > bookmark]

        # If start_indices contains multiple idxs, get start_idx from front of list (first occurance) and save rest to repeat_mentions
        # If start_indices contains just one idx, that is the start_idx
        if len(start_indices) > 0:
            start_idx = start_indices[0]
            end_idx = start_idx + len(mention.split()) - 1
            mention_span = [idxs[start_idx],idxs[end_idx]]

            bookmark = end_idx

        mention_spans.append(mention_span)

    return mention_spans

In [109]:
words = faa['19820725041999I']
mentions = ['LOOSE COWLING ON TAKEOFF','COWLING']
get_spans(mentions, words)

[[0, 3], [5, 5]]

In [110]:
# Iterate through c119_coref_chains

formatted = []

for i in range(len(result_df)):

    coref_chains = c119_coref_chains[i]

    output_chains = []
    for coref_chain in coref_chains:
        
            # Get spans of words of coref chain using get_spans
            chain_spans = get_spans(coref_chain, faa[result_df['c5'][i]])

            output_chains.append(chain_spans)

    formatted.append(output_chains)

formatted[2320:2330] # sample output

[[[[3, 5], [16, 17]]], [], [], [], [], [], [], [], [], []]

**Check for errors**

In [111]:
errs = []
for iresult, result in enumerate(formatted):
    for chain in result:
        if [-1,-1] in chain: # should also look for [-2, *]
            errs.append(iresult)

In [112]:
sample = pd.read_csv('../../OMIn_dataset/data/FAA_data/FAA_sample_100.csv')
errs_to_check = []
for err in errs:
    if err in sample['Unnamed: 0']:
        errs_to_check.append(err)

In [113]:
errs_to_check # No errors occur within the data sampled for evaluation

[]

**Add formatted column to df**

In [114]:
result_df['corefs'] = formatted

In [115]:
result_df[result_df['c5'] == '19990213001379A'] # sample output

,c5,c119,c119_coref,c119_ner,c119_pos,c119_lemmatized,c119_dependency,c119_sentiment,corefs
2318,19990213001379A,ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CON...,[],"[('ACFT', 'PERSON'), ('RAN INTO', 'ORG'), ('TR...","[('ACFT', 'PROPN'), ('WAS', 'VERB'), ('TAXIING...","['ACFT', 'be', 'taxi', 'for', 'take', 'off', '...","[('ACFT', 'nsubj', 'TAXIING'), ('WAS', 'auxpas...",-0.125,[]


In [116]:
result_df.to_csv('../../tool_results/neuralcoref/neuralcoref_sm.csv', index=False)